In [13]:
import os 
import requests
import json 
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from web.webRead import Website

In [14]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You ara able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an about page, or a company page, or carrer/Jobs page.\n"
link_system_prompt += "You should repons in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"}
        {"type": "carrers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
def get_links_user_prompt(Website):
    user_prompt = f"Here is the list of links on the website of {Website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(Website.links)
    return user_prompt

In [16]:
load_dotenv(r'C:\Users\Gabriel\Documents\key\key.env')
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found")
else:
    print("API key found")
openai = OpenAI()

API key found


In [17]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        
        model = 'gpt-4o-mini',
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format = {"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [18]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}